# JetRacer - Data collection

In this notebook we'll collect training data for CNN VAE. The training data save to dataset directory.

## Import module


In [1]:
import os
import cv2
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display
from jetcam.csi_camera import CSICamera
from jetcam.utils import bgr8_to_jpeg

## Initialize Camera

Next is initializing camera module. Image size is 320 x 240. Frame rate is about 60Hz. We'll save image in camera observer method. camera observer method can get image per frame rate. Thus, frame rate is decide to image save interval.

In [6]:
CAMERA_WIDTH = 320
CAMERA_HEIGHT = 240

camera = CSICamera(width=CAMERA_WIDTH, height=CAMERA_HEIGHT, capture_width=CAMERA_WIDTH, 
                   capture_height=CAMERA_HEIGHT, capture_fps=60)

camera.running = True

image = widgets.Image(format='jpeg', width=320, height=240)
camera_link = traitlets.dlink((camera,'value'), (image,'value'), transform=bgr8_to_jpeg)
display(image)

## UI Widget

We can check Gamepad value and Image. 

In [7]:
from IPython.display import display
import ipywidgets

log_button = ipywidgets.ToggleButton(value=False, description='enable logging')
display(log_button)


Directories not created becasue they already exist


## Set callback for collect the training data.

```execute``` is callback for training data. The method set to camera observer. This callback saving the image to DATASET_DIR. When click ```enable logging``` button, this method recording training data. You can check number of training data with ```Number image text box```.

In [8]:
DATASET_DIR = 'dataset'

if not os.path.exists(DATASET_DIR):
    os.mkdir(DATASET_DIR)

count = 0

def execute(change):
    global count, DATASET_DIR
    image = change['new']
    if log_button.value:
        image_path = os.path.join(DATASET_DIR, '%012d.jpg' % count)
        cv2.imwrite(image_path, image)
        count += 1
    else:
        count = 0

execute({'new': camera.value})
camera.observe(execute, names='value')

## Cleanup
After collecting enough data. cleanup camera observer .

In [ ]:
camera.unobserve_all()

## Cleate dataset.zip file 

In [10]:
import datetime
def timestr():
    return str(datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S'))

!zip -r -q jetracer_{DATASET_DIR}_{timestr()}.zip {DATASET_DIR}